# Homework 4 Submission
## Yonadav Shavit

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.special import factorial
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(Let's define some latex commands)
 
$\newcommand{\floor}[1]{\left \lfloor #1 \right \rfloor}$
$\newcommand{\clamp}[1]{\left [ #1 \right ]}$
$\newcommand{\lap}[1]{Lap\left ( #1 \right )}$
$\newcommand{\lapcdf}[2]{LapCDF\left ( #1, [-\inf, #2] \right )}$
$\newcommand{\lappdf}[2]{LapPDF\left ( #1, |_{#2} \right )}$
$\newcommand{\lapcdffull}[2]{\frac{1}{2} + \frac{1}{2}sgn\left({#2}\right)\left ( 1 - \exp{\left ( - \frac{|#2|}{#1} \right ) }\right )}$
$\newcommand{\lappdffull}[2]{\frac{1}{2 #1} \exp{\left ( - \frac{#2}{#1}\right ) }}$

## Problem 1

### (a)

We'll be implementing two different mechanisms, both of which rely on an estimation of $p_j$.

In the local DP case, we'll compute $I(x[j] = 0 \land y = 1)$, and then return that value (or its conjugate) with probability $\frac{e^\epsilon}{1 + e^\epsilon}$, and then scale that up by a factor of $c_\epsilon = \frac{e^\epsilon + 1}{e^\epsilon - 1}$.

In the centralized DP case, we'll compute $\frac{1}{n}\sum_{i = 0}^n{I(x_i[j] = 0 \land y_i = 1)}$, and then add Laplace noise of magnitude $\frac{1}{n\epsilon}$.

In [3]:
df = pd.read_csv("../data/hw4testdata.csv")

In [9]:
X_toy = df[[col for col in df.columns if "x" in col]]
Y_toy = df['y']

In [11]:
def local_pj(Xj, Y, epsilon):
    n = Xj.shape[0]
    eeps = e**epsilon
    correct_vals = Xj == Y
    randomize = np.random.binomial(1, eeps/(1+eeps), size=n).astype(bool)
    randomized_vals = np.logical_xor(randomize, correct_vals)
    c_e = (eeps + 1)/(eeps - 1)
    return c_e*randomized_vals.mean()

def lap_mechanism(epsilon, GS_q, size=None):
    return np.random.laplace(scale=GS_q/epsilon, size=size)

def dp_mean(x, epsilon, b, a=0):
    x_clipped = np.clip(x, a, b)
    GS_q = np.abs(b-a)/x.shape[0]
    return x_clipped.mean() + lap_mechanism(epsilon, GS_q)

def global_pj(Xj, Y, epsilon):
    return dp_mean((Xj==Y).astype(int), epsilon, b=1)

def sq_mechanism(X, Y, epsilon, t, pj_alg):
    d = X.shape[1]
    ps = np.array([pj_alg(X[:, j], Y, epsilon/d) for j in range(d)])
    S = (ps < t).astype(int)
    return S

local_sq_mechanism = lambda *args: sq_mechanism(*args, local_pj)
global_sq_mechanism = lambda *args: sq_mechanism(*args, global_pj)

def compute_fpr_and_fnr(S, X, Y):
    Yhat = np.prod(S*X, axis=1)
    fpr = Yhat*(1-Y)
    fnr = (1-Yhat)*Y
    return fpr, fnr

In [ ]:
local_S = local_sq_mechanism(X_toy, Y_toy, epsilon=1, t=0.5)
global_S = global_sq_mechanism(X_toy, Y_toy, epsilon=1, t=0.5)

# need to validate the method still?

## (b)


We will leverage the following approximation to the binomial distribution, which holds under large $n$:
$$f(x)=
\frac{n !}{x !(n-x) !} p^{x} q^{n-x} \approx 
\mathcal{N}(np, npq) = 
\frac{1}{\sqrt{2 \pi n p q}} e^{-(x-n p)^{2} / 2 n p q} $$

First, we'll think about the local model.
Let's examine the probability that $p_j$ is included in $\hat{S}$ but not $S$ (i.e. false inclusion):


Examine the probability of false exclusion

Combine into total error by addition + union bound

Then do false positives in global model

False negatives in global model

combine total error by addition + union bound

Probably do this for each individual j and then show the union bound

## (c)
